In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyECLAT import ECLAT
import time

**Descripción general del algoritmo ECLAT**

El algoritmo ECLAT (Agrupación de clases de equivalencia y recorrido de celosía ascendente) es un algoritmo de minería de datos para la minería de reglas de asociación mientras resuelve el problema de análisis de la canasta del cliente: el objetivo es comprender qué productos se compran a menudo juntos. 

El algoritmo ECLAT no se puede aplicar a los datos representados en formato horizontal y debe convertirlos al formato vertical antes de usarlo. Este enfoque vertical del algoritmo ECLAT lo hace más rápido que los algoritmos de crecimiento Apriori y FP, ya que escanea la base de datos solo una vez. El algoritmo Apriori escanea la base de datos en cada iteración, y el algoritmo de crecimiento FP lo hace dos veces.


El algoritmo ECLAT se aplicará al conjunto de datos para descubrir conjuntos de películas de Marvel que frecuentemente son **calificadas positivamente** (para ello el corte será a partir de una calificación de 6) y vistas juntas. Al convertir los datos a un formato vertical, ECLAT identificará eficientemente estas asociaciones, utilizando los identificadores de usuarios como transacciones y las películas como ítems. Esta información permitirá desarrollar un sistema de recomendación que sugiera a los usuarios qué películas ver a continuación, basándose en patrones de visualización y gusto similares observados en otros usuarios. Esto ayudará a personalizar la experiencia de streaming, mejorando las recomendaciones y potencialmente incrementando la satisfacción y retención de los usuarios.


In [20]:
data_apriori = pd.read_csv('data/data_apriori.csv')
data_apriori_bin = pd.read_csv('data/data_apriori_bi.csv')
#data_apriori = data_apriori.astype(str)
data_apriori.head()

,Iron Man,The Incredible Hulk,Iron Man 2,Thor,Captain America: The First Avenger,The Avengers,Iron Man 3,Thor: The Dark World,Captain America: The Winter Soldier,Guardians of Galaxy,...,Black Panther,Avengers: Infinity War,Ant-Man and the Wasp,Captain Marvel,Avengers: Endgame,Spider-Man: Far From Home,Black Widow,Shang-Chi and the Legend of the Ten Rings,Eternals,Spider-Man: No Way Home
0,4.0,2.0,10.0,0.0,6.0,0.0,2.0,9.0,NaN,6.0,...,4.0,8.0,9.0,9.0,NaN,10.0,8.0,10.0,1.0,10.0
1,NaN,9.0,0.0,10.0,NaN,NaN,1.0,10.0,10.0,2.0,...,9.0,4.0,NaN,9.0,9.0,10.0,9.0,10.0,1.0,10.0
2,10.0,NaN,10.0,NaN,NaN,4.0,10.0,NaN,NaN,4.0,...,9.0,2.0,7.0,4.0,8.0,6.0,NaN,4.0,NaN,2.0
3,10.0,5.0,7.0,NaN,NaN,0.0,NaN,9.0,10.0,6.0,...,0.0,9.0,4.0,NaN,9.0,10.0,NaN,9.0,NaN,10.0
4,10.0,2.0,10.0,2.0,4.0,1.0,10.0,0.0,1.0,2.0,...,NaN,8.0,NaN,8.0,2.0,2.0,4.0,1.0,9.0,7.0


In [21]:
data_apriori_bin

,Iron Man,The Incredible Hulk,Iron Man 2,Thor,Captain America: The First Avenger,The Avengers,Iron Man 3,Thor: The Dark World,Captain America: The Winter Soldier,Guardians of Galaxy,...,Black Panther,Avengers: Infinity War,Ant-Man and the Wasp,Captain Marvel,Avengers: Endgame,Spider-Man: Far From Home,Black Widow,Shang-Chi and the Legend of the Ten Rings,Eternals,Spider-Man: No Way Home
0,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,0,1,1,1,1,1
1,0,1,1,1,0,0,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
2,1,0,1,0,0,1,1,0,0,1,...,1,1,1,1,1,1,0,1,0,1
3,1,1,1,0,0,1,0,1,1,1,...,1,1,1,0,1,1,0,1,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29123,1,1,0,1,1,0,0,1,1,1,...,1,0,1,0,1,0,1,1,1,1
29124,0,1,1,0,0,1,1,0,1,1,...,1,0,0,0,1,1,1,1,0,1
29125,1,1,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,0,1,1,1
29126,1,1,0,1,1,0,1,1,0,1,...,1,1,1,0,0,1,0,1,1,0


In [27]:
import pandas as pd

data_apriori_bin = (data_apriori > 0).astype(int)

# Lista para almacenar cada fila nueva antes de concatenar al DataFrame
rows_list = []

for movie in data_apriori_bin.columns:
    users_who_rated = data_apriori_bin.index[data_apriori_bin[movie] == 1].tolist()
    tid_set = ','.join(map(str, users_who_rated))
    rows_list.append({'Itemset': movie, 'TID_set': tid_set})

# Crear DataFrame desde la lista de diccionarios
vertical_format = pd.DataFrame(rows_list)

# Mostrar el DataFrame en formato vertical
print(vertical_format)





                                      Itemset  \
0                                    Iron Man   
1                         The Incredible Hulk   
2                                  Iron Man 2   
3                                        Thor   
4          Captain America: The First Avenger   
5                                The Avengers   
6                                  Iron Man 3   
7                        Thor: The Dark World   
8         Captain America: The Winter Soldier   
9                         Guardians of Galaxy   
10                    Avengers: Age of Ultron   
11                                    Ant-Man   
12                 Captain America: Civil War   
13                             Doctor Strange   
14                      Guardians of Galaxy 2   
15                     Spider-Man: Homecoming   
16                             Thor: Ragnarok   
17                              Black Panther   
18                     Avengers: Infinity War   
19                  

In [28]:

#vertical_format.columns = range(data_apriori.shape[0])

# Crear una instancia de ECLAT
try:    
    start_time = time.time()
    eclat_instance = ECLAT(data=data_apriori, verbose=True)
    print("ECLAT instance created successfully.")
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution Time for sample eclat_instance : {execution_time} seconds\n")
except Exception as e:
    print("Failed to create ECLAT instance:", e)

eclat_instance.df_bin.head().T

  0%|          | 0/38 [00:00<?, ?it/s]

Failed to create ECLAT instance: 0


NameError: name 'eclat_instance' is not defined

Vamos a ver si se ha vosto o no se ha visto, luego el usuario podrá dar el voto que sea, pero nos interesa saber si se ha visto o no.


En este conjunto de datos binarios, cada fila representan identificadores de los usuarios. Cada columna representa una puntuación posible de 0 a 10. Estas columnas son entonces agrupadas por el tipo de puntuación que un usuario ha dado a cualquier película, no sólo una película específica. Por ejemplo, la columna '9.0' en la imagen 2 muestra en forma binaria (1 o 0) si el usuario ha dado una puntuación de 9 a cualquier película.

0 – el usuario no votó esa nota para ninguna película.
1 – el usuario votó esa nota para alguna película.

Ahora, necesitamos contar los elementos de cada columna del DataFrame:

In [ ]:
items_total = eclat_instance.df_bin.astype(int).sum(axis=0)        # count items in each column
items_per_instans = eclat_instance.df_bin.astype(int).sum(axis=1)  # count items in each row
# Loading items per column stats to the DataFrame
df = pd.DataFrame({'items': items_total.index, 'transactions': items_total.values}) 
# cloning pandas DataFrame for visualization purpose  
df_table = df.sort_values("transactions", ascending=False)
#  Top 5 most popular products/items
df_table.style.background_gradient(cmap='Blues')

NameError: name 'eclat_instance' is not defined

In [ ]:
items_per_instans

0         9
1         8
2        10
3         9
4        10
         ..
29123     7
29124     7
29125    11
29126    11
29127     8
Length: 29128, dtype: int64

In [ ]:
min_support = 10/100                     # el voto debe aparecer al menos en el 10% de las puntuacion
min_combination = 2                      # comenzar que contengan al menos 2 items
max_combination = max(items_per_instans) # hasta un máximo de votos por puntuación

try:
    # El método fit devuelve los índices de las transacciones y los soportes de itemsets frecuentes
    print("Running ECLAT...")
    start_time = time.time()
    get_ECLAT_indexes, get_ECLAT_supports = eclat_instance.fit(min_support=min_support,  
                                                                min_combination=min_combination,
                                                                max_combination=max(items_per_instans),
                                                                separator=' & ',
                                                                verbose=True)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution Time for sample eclat_instance : {execution_time} seconds\n")
except Exception as e:
    print(f"Failed to generate rules: {e}")


Running ECLAT...
Combination 2 by 2


55it [00:00, 88.35it/s]


Combination 3 by 3


165it [00:01, 91.19it/s]


Combination 4 by 4


...

Combination 10 by 10


11it [00:00, 119.85it/s]


Combination 11 by 11


1it [00:00, 104.42it/s]


Execution Time for sample eclat_instance : 25.07173204421997 seconds



In [ ]:
# Ordenar los resultados de ECLAT por soporte
sorted_supports = sorted(get_ECLAT_supports.items(), key=lambda x: x[1], reverse=True)

# Crear un DataFrame con los resultados ordenados
df = pd.DataFrame(sorted_supports, columns=['Item Set', 'Support'])

# Mostrar los primeros 5 resultados con un gradiente de colores para resaltar los soportes más altos
df.head().style.background_gradient(cmap='Blues')

,Item Set,Support
0,8.0 & 10.0,0.794459
1,0.0 & 10.0,0.794116
2,4.0 & 10.0,0.794116
3,10.0 & 2.0,0.793704
4,3.0 & 10.0,0.793017


https://hands-on.cloud/implementation-of-eclat-algorithm-using-python/

https://pypi.org/project/pyECLAT/
